In [ ]:
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 11.6 MB/s eta 0:00:00


In [ ]:
from Bio.Align import substitution_matrices
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
import numpy as np

/usr/local/lib/python3.10/dist-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [ ]:
matrix = substitution_matrices.load('BLOSUM62')
matrix.alphabet


'ARNDCQEGHILKMFPSTWYVBZX*'

#Нидлман Вунш (3 балла)


Реализуйте алгоритм Нидлмана Вунша для выравнивания последовательностеей. На вход принимается две строки, матрица замен и стоимость гэпа. В результате верните оптимальное выравнивание и его вес. При проверке помните, что оптимальных выравниваний может быть несколько, но вес у них должен совпадать.

In [118]:
def Needleman_Wunsch(s1, s2, matrix, gap_w):

    # Для быстрой работы с матрицей весов в формате как у 'BLOSUM62'
    # занесем alphabet в словарь alfab

    alfab = {char: i for i, char in enumerate(matrix.alphabet)}
    n, m = len(s1), len(s2)
    dp = np.vstack([np.array([0+i*gap_w for i in range(n+1)]), np.zeros((m, n+1))])
    vector = np.zeros((m+1, n+1))
    vector[:, 0] = 1
    vector[0] = 2
    for i in range(1, m+1):
      dp[i, 0] = dp[i-1, 0] + gap_w
      for j in range(1, n+1):
          a, b = alfab[s2[i-1]], alfab[s1[j-1]]
          dp[i][j] = max(dp[i-1, j] + gap_w,
                         dp[i, j-1] + gap_w,
                         dp[i-1, j-1] + matrix[a][b])
          if dp[i, j] == dp[i-1, j] + gap_w:
                vector[i, j] = 1  # up
          elif dp[i, j] == dp[i, j-1] + gap_w:
                vector[i, j] = 2  # left
          else:
                vector[i, j] = 0  # giag


    aligned_s1 = []
    aligned_s2 = []
    view = []
    i, j = m, n
    while i > 0 or j > 0:
        if vector[i, j] == 0:
            aligned_s2.append(s2[i-1])
            aligned_s1.append(s1[j-1])
            if s2[i-1] == s1[j-1]:
                view += '|'
            else:
                view += '.'
            i -= 1
            j -= 1
        elif vector[i, j] == 1:
            aligned_s2.append(s2[i-1])
            aligned_s1.append('-')
            view += ' '
            i -= 1
        else:
            aligned_s2.append('-')
            aligned_s1.append(s1[j-1])
            view += ' '
            j -= 1
    aligned_s1.reverse()
    aligned_s2.reverse()
    return (aligned_s1, aligned_s2, view[::-1]), dp[-1, -1]

Проверка:

In [119]:
gap_penalty = -3
substitution_matrix = substitution_matrices.load("BLOSUM62")

seq1 = "ACGTAZWEDFDDD"
seq2 = "ACTTAADFQQ"

In [120]:
alignments = alignments = pairwise2.align.globalds(seq1, seq2, substitution_matrix, gap_penalty, gap_penalty)

print(format_alignment(*alignments[0]))

ACGTAZWEDFDDD
||.||  .|| ..
ACTTA--ADF-QQ
  Score=22



In [121]:
(s1, s2, v), score = Needleman_Wunsch(seq1, seq2, substitution_matrix, gap_penalty)
print(*s1, sep='')
print(*v, sep='')
print(*s2, sep='')
print('Score=', int(score))


ACGTAZWEDFDDD
||.||.  ||.. 
ACTTAA--DFQQ-
Score= 22


#Афинные гэпы (4 балла)

Афинные гэпы (4 балла)
Реализуйте выравнивание с афинными гэпами, алгоритм на вход принимает две строки, матрицу замен, штраф за начало гэпа α, и за его продолжение β. В результате возвращает выравнивание и его вес. Сложность алгоритма квадратичная по памяти и по времени.

In [ ]:
def Affine_gap(s1, s2, matrix, alpha, beta):

    # Для быстрой работы с матрицей весов в формате как у 'BLOSUM62'
    # занесем alphabet в словарь alfab

    alfab = {char: i for i, char in enumerate(matrix.alphabet)}
    n, m = len(s1), len(s2)

    # Начальные условия для динамики
    dp = np.vstack([np.array([0]+[alpha + (i-1)*beta for i in range(1, n+1)]), np.zeros((m, n+1))])
    #dp = np.zeros((m+1, n+1))
    gap_v = np.zeros((m+1, n+1))
    gap_v[1:, 0] = [alpha + (i-1)*beta for i in range(1, m+1)]
    gap_h = np.zeros((m+1, n+1))
    gap_h[0, 1:] =  [alpha + (i-1)*beta for i in range(1, n+1)]

    for i in range(1, m+1):
      dp[i, 0] = alpha + (i-1)*beta
      for j in range(1, n+1):
          a, b = alfab[s2[i-1]], alfab[s1[j-1]]
          # gap_v[i][j] = max(gap_v[i-1][j] + beta, dp[i-1][j] + alpha )
          # gap_h[i][j] = max(gap_h[i][j-1] + beta, dp[i][j-1] + alpha )

          gap_v[i][j] = max(gap_v[i][j-1] + beta, dp[i][j-1] + alpha )
          gap_h[i][j] = max(gap_h[i-1][j] + beta, dp[i-1][j] + alpha )
          mat = dp[i-1][j-1] + matrix[a][b]

          dp[i][j] = max(mat, gap_v[i][j], gap_h[i][j])


    align1 = ""
    align2 = ""
    view = ""
    i, j = m, n

    while i > 0 or j > 0:
        if i > 0 and dp[i][j] == gap_v[i][j]:
            align2 = s2[i-1] + align2
            align1 = "-" + align1
            view = ' ' + view
            i -= 1
        elif i > 0 and j > 0 and dp[i][j] == dp[i-1][j-1] + matrix[s2[i-1]][s1[j-1]]:
            align2 = s2[i-1] + align2
            align1 = s1[j-1] + align1
            if s2[i-1] == s1[j-1]:
                view = '|' + view
            else:
                view = '.' + view
            i -= 1
            j -= 1
        else:
            align2 = "-" + align2
            align1 = s1[j-1] + align1
            view = ' ' + view
            j -= 1
    #print(dp)
    # print(gap_v)
    # print(gap_h)
    return (align1, align2, view), dp[-1, -1]



Проверка

In [ ]:
seq1 = "MMMFRERRRY"
seq2 = "MNFRY"

alpha, beta = -10, -1
substitution_matrix = substitution_matrices.load("BLOSUM62")

In [ ]:
alignments = pairwise2.align.globalds(seq1, seq2, substitution_matrix, alpha, beta)
print(format_alignment(*alignments[0]))

MMMFRERRRY
|..     ||
MNF-----RY
  Score=1



In [ ]:
(s1, s2, v), score = Affine_gap(seq1, seq2, substitution_matrix, alpha, beta)
print(*s1, sep='')
print(*v, sep='')
print(*s2, sep='')
print('Score=', int(score))

MMMFRER-R-RY
        . ||
-------MNFRY
Score= 1


In [ ]:
seq1='GATTACA'
seq2= 'CATTAGA'
alpha, beta = -10, -0.5

In [ ]:
alignments = pairwise2.align.globalds(seq1, seq2, substitution_matrix, alpha, beta)
print(format_alignment(*alignments[0]))

GATTACA
.||||.|
CATTAGA
  Score=16



In [ ]:
(s1, s2, v), score = Affine_gap(seq1, seq2, substitution_matrix, alpha, beta)
print(*s1, sep='')
print(*v, sep='')
print(*s2, sep='')
print('Score=', int(score))

GATTACA
.||||.|
CATTAGA
Score= 16


#Количество выравниваний (4 балла)
Выведите рекуррентную формулу количества всех возможных выравниваний последовательностей длины n и m пользуясь разбиением всех выравниваний на непересекающиеся блоки. (1.5 балл)
Получите точную формулу, основываясь на начальные условия и рекуррентную формулу. (1.5 балл)
Воспользуйтесь приближением Стирлинга чтобы получить приближенную формулу количества выравниваний. (1)

Рекуррентная формула количества всех возможных выравниваний для последовательностей длины n и m можно выразить следующим образом:

F(n, m) = F(n-1, m) + F(n, m-1) + F(n-1, m-1)

где F(n, m) - количество всех возможных выравниваний для последовательностей длины n и m.

Точная формула может быть получена из начальных условий F(0, 0) = 1, F(0, m) = 1, F(n, 0) = 1, и рекуррентной формулы выше, как решение диференциального уравнения.



$F(n, m) = \displaystyle\sum_{k=0}^{min(m,n)} 2^{k} \binom{n}{k}\binom{m}{k}$

Формула Стирлинга

$n! ≈ sqrt(2πn) * (n/e)^n$



$F(n, m) = \displaystyle\sum_{k=0}^{min(m,n)} 2^{k+1}π \sqrt{\frac{nm}{k^2(n-k)(m-k)} }\frac{n^nm^m}{k^{2k}(n-k)^{n-k}(m-k)^{m-k}}$